Top accuracy: 82.26%
<hr>
Epochs: 130

```python

optimizer = optim.Adam(net.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3)
        self.conv2 = nn.Conv2d(64, 128, 3)
        self.conv3 = nn.Conv2d(128, 256, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 2 * 2, 512)  
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

```


In [35]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print("HIP version:", torch.version.hip)
print("Device count:", torch.cuda.device_count())
print("Device 0:", torch.cuda.get_device_name(0))

Using device: cuda
HIP version: 6.2.41133-dd7f95766
Device count: 1
Device 0: AMD Radeon RX 6800


In [36]:
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
import torchvision
import torchvision.transforms as transforms

In [37]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [49]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Podzielmy testowy zbiór na walidacyjny i testowy
val_size = 5000
test_size = len(test_data) - val_size

val_data, test_data = random_split(test_data, [val_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

print(f'Training dataset: {len(train_loader.dataset)}')
print(f'Validation dataset: {len(val_loader.dataset)}')
print(f'Test dataset: {len(test_loader.dataset)}')


Files already downloaded and verified
Files already downloaded and verified
Training dataset: 50000
Validation dataset: 5000
Test dataset: 5000


In [39]:
image, label = train_data[0]

In [40]:
image.size()

torch.Size([3, 32, 32])

In [41]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [42]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3)
        self.conv2 = nn.Conv2d(64, 128, 3)
        self.conv3 = nn.Conv2d(128, 256, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 2 * 2, 512)  
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [43]:
net = NeuralNet().to(device)
loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

torch.manual_seed(42)

In [44]:

num_of_epochs = 130

highest_val_accuracy = 0.0

for epoch in range(num_of_epochs):
    net.train()
    
    running_loss = 0.0

    # Trening
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = loss_function(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    net.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total

    scheduler.step()

    if val_accuracy > highest_val_accuracy:
        highest_val_accuracy = val_accuracy

    print(f'Epoch ({epoch + 1}/{num_of_epochs}), '
          f'Train Loss: {avg_train_loss:.4f}, '
          f'Val Loss: {avg_val_loss:.4f}, '
          f'Val Accuracy: {val_accuracy:.2f}%')

print(f'Highest Validation Accuracy: {highest_val_accuracy:.2f}%')

Epoch (1/130), Train Loss: 1.5398, Val Loss: 1.1832, Val Accuracy: 57.02%
Epoch (2/130), Train Loss: 1.1594, Val Loss: 1.0258, Val Accuracy: 63.42%
Epoch (3/130), Train Loss: 1.0088, Val Loss: 0.9366, Val Accuracy: 66.60%
Epoch (4/130), Train Loss: 0.9145, Val Loss: 0.8751, Val Accuracy: 69.40%
Epoch (5/130), Train Loss: 0.8580, Val Loss: 0.8673, Val Accuracy: 69.40%
Epoch (6/130), Train Loss: 0.8032, Val Loss: 0.8018, Val Accuracy: 72.24%
Epoch (7/130), Train Loss: 0.7725, Val Loss: 0.7786, Val Accuracy: 73.38%
Epoch (8/130), Train Loss: 0.7396, Val Loss: 0.8093, Val Accuracy: 72.20%
Epoch (9/130), Train Loss: 0.7192, Val Loss: 0.7589, Val Accuracy: 73.70%
Epoch (10/130), Train Loss: 0.7014, Val Loss: 0.7715, Val Accuracy: 73.18%
Epoch (11/130), Train Loss: 0.6791, Val Loss: 0.7073, Val Accuracy: 76.52%
Epoch (12/130), Train Loss: 0.6636, Val Loss: 0.7399, Val Accuracy: 74.60%
Epoch (13/130), Train Loss: 0.6548, Val Loss: 0.6998, Val Accuracy: 75.82%
Epoch (14/130), Train Loss: 0.6376

In [45]:
torch.save(net.state_dict(), 'trained_model.pth')

In [46]:
net = NeuralNet().to(device)
net.load_state_dict(torch.load('trained_model.pth'))

/tmp/ipykernel_152461/838394504.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_model.pth'))


<All keys matched successfully>

In [47]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 80.38%


In [ ]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['data/tests/ship.jpg', 'data/tests/cat.jpg', 'data/tests/dog.jpg']
images = [load_image(path).to(device) for path in image_paths]

net.eval()
with torch.no_grad():
    for i, image in enumerate(images):
        outputs = net(image)
        _, predicted = torch.max(outputs, 1)
        print(f'Image: {image_paths[i]}, Predicted class: {class_names[predicted.item()]}')

Image: ship.jpg, Predicted class: ship
Image: cat.jpg, Predicted class: cat
Image: dog.jpg, Predicted class: dog
